# Probando Tensorflow

## MNIST (beginners)

Script para descargar los datos de MNIST

In [1]:
import tensorflow as tf
import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Vemos que los datos se dividen en 2 matrices, una de datos para entrenar (mnist.train) y otra para probar nuestro
aprendizaje (mnist.test).
La matriz train cuenta con 55000 imagenes de 28x28 pixeles, es decir, 55000 vectores de tamano 784 que representan
imagenes de digitos escritos a mano. La matriz `mnist.train.label` es la que indica que digito es cada imagen.

In [2]:
print "Dimensiones de la matriz de imagenes >> ", mnist.train.images.shape
print "Dimension de la matriz de labels >> ", mnist.train.labels.shape
print "Dimensiones de la matriz de prueba >> ", mnist.test.images.shape

Dimensiones de la matriz de imagenes >>  (55000, 784)
Dimension de la matriz de labels >>  (55000, 10)
Dimensiones de la matriz de prueba >>  (10000, 784)


In [3]:
# x es solo un placeholder, es decir no tiene un valor especifico aun
x = tf.placeholder("float", [None, 784])   # La dimension de x, es (lo que sea, 784)
W = tf.Variable(tf.zeros([784,10]))        # Pesos
b = tf.Variable(tf.zeros([10]))            # Bias
y = tf.nn.softmax(tf.matmul(x,W) + b)      # Predicted probability

y_ = tf.placeholder("float", [None,10]) 
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)    # Minimizamos con descenso de
# gradiente, con un grado de aprendizaje de 0.01

# Inicializamos las variables que creamos
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

# Entrenamos con 1000 iteraciones
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))    # Acertamos ?

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))   # Con que exactitud ?

print "Presicion con nuestros datos de entrenamiento >> ", sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

Presicion con nuestros datos de entrenamiento >>  0.9189


# Deep MINST

In [4]:
# Tensorflow utiliza un backend en C++ para hacer el computa mas eficiente.
# Para establecer una coneccion con este backend usaremos una sesion.
sess = tf.InteractiveSession()


# Nodos para las imagenes de entrada y las clases de salida objetivo
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

# pesos y bias ('tensors' de puros ceros hasta ahora)
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# inicializamos las variables previas definidas para poder usarlas
# se inician dentro de la sesion que creamos
sess.run(tf.initialize_all_variables())

# ahora implementamos nuestro modelo de regresion (softmax)
y = tf.nn.softmax(tf.matmul(x,W) + b)

# especificamos nuestra funcion costo a minimizar, la cual sera
# 'cross-entropy' entre los objetivos y nuestra prediccion de modelo
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# usamos descenso de gradiente para minimizar la entropia y repetimos
# esto con un epoc de 1000
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# checamos que tan bien le fue a nuestro modelo comparando
# la etiqueta que nuestro modelo dice es la mas probable para cada entrada
# con la etiqueta correcta.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# como 'correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))' nos
# dara una lista de booleanos, convertiremos la lista a 0's y 1's y 
# despues le sacamos la media
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# ahora evaluamos nuestra precision con los datos de prueba
print "Precision con nuestros datos de prueba >> ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

Precision con nuestros datos de prueba >>  0.9139


## Multilayer Convolutional Network

In [5]:
# esta funcion regresa una matriz con pesos iniciales
# aleatorios provenientes de una distribucion normal
# con desviacion estandar = .1
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# esta funcion regresa una matriz con todos los elementos
# igual a .1 y dimensiones = shape
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


Definimos nuestras funciones de convolucion y pooling maximo de bloques de 2x2. Las usaremos mas adelante

In [6]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


### Primera capa convolucional

En esta capa se procesan 32 caracteristicas por cada pieza de datos de 5x5.

Tambien creamos un vector de bias por cada salida (de las 32). Para esto usamos las funciones creadas anteriormente

In [7]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

Le hacemos reshape a x, haciendo un tensor 4d. La segunda y tercera dimension corresponden al ancho y alto de la imagen.

In [8]:
x_image = tf.reshape(x, [-1,28,28,1])

Se aplica la funcion rectificadora al vector x convolucionado con el vector de pesos, al mismo tiempo se le anade el ruido.

Despues aplicamos el max pooling sobre una region de 2x2

In [9]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Creando otra capa mas

Agregamos otra capa mas de convolucion para crear la red profunda.

Esta capa tendra 64 caracteristicas por cada bloque de 5x5

In [10]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Ahora creamos una capota completamente conectada, con 1024 neuronas que procesaran toda la imagen. Primero creamos el vector de pesos con la imagen reducida a 7x7, luego creamos el vector con los bias. Redimensionamos el tensor de la capa de pooling en un "lote" de vectores. Por ultimo lo multiplicamos por la matriz de pesos (anadiendole el bias), y a eso le aplicamos la funcion de activacion rectificadora. 

In [11]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Para reducir el sobre-aprendizaje, aplicaremos algo como regularizacion en redes neuronales, un concepto bastante interesante, el "dropout", que es, en algunos casos, poniendo algunas unidades de activacion en cierta capa a cero.

In [12]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Finalmente creamos una capa mas con una regresion softmax, con sus respectivos pesos y bias

In [13]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


### Entrenamiento y evaluacion del model

Para entrenar y evaluar nuestro modelo usaremos codigo muy parecido al de la regresion Softmax pasada, la diferencia es que reemplazaremos el descenso de gradiente con un optimizador mas sofisticado, ADAM, incluimos tambien `keep_prob` dentro del diccionario `feed_dict` para controlar la tasa de dropout. Cada 100 iteraciones calculamos la precision de entrenamiento.

In [15]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("Iteracion %d, precision de entrenamiento %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("Exactitud de prueba %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Iteracion 0, precision de entrenamiento 0.16
Iteracion 100, precision de entrenamiento 0.9
Iteracion 200, precision de entrenamiento 0.8
Iteracion 300, precision de entrenamiento 0.96
Iteracion 400, precision de entrenamiento 0.96
Iteracion 500, precision de entrenamiento 0.98
Iteracion 600, precision de entrenamiento 0.98
Iteracion 700, precision de entrenamiento 0.98
Iteracion 800, precision de entrenamiento 0.96
Iteracion 900, precision de entrenamiento 0.94
Iteracion 1000, precision de entrenamiento 0.98
Iteracion 1100, precision de entrenamiento 0.96
Iteracion 1200, precision de entrenamiento 0.96
Iteracion 1300, precision de entrenamiento 0.98
Iteracion 1400, precision de entrenamiento 0.94
Iteracion 1500, precision de entrenamiento 1
Iteracion 1600, precision de entrenamiento 0.96
Iteracion 1700, precision de entrenamiento 1
Iteracion 1800, precision de entrenamiento 0.96
Iteracion 1900, precision de entrenamiento 0.96
Iteracion 2000, precision de entrenamiento 0.96
Iteracion 21